In [64]:
import pandas as pd
import numpy as np
import re
import json
from pathlib import Path

O formato em que as informações estão dispostas no arquivo `PNS_2019.txt` seguindo o formato fixed-width:
- Cada linha representa uma entrada diferente,
- O arquivo `dicionario_PNS_microdados_2019.xls` documenta as variáveis e em qual posição estão.

Vamos começar coletando as informações do dicionário.

In [65]:
df = pd.read_excel("dicionario_PNS_microdados_2019.xls", skiprows=1, header=0)

Vamos validar as colunas e colocar os nomes corretos:

In [66]:
print(df.columns)

Index(['Posição inicial ', 'Tamanho', 'Código\nda\nvariável', 'Quesito',
       'Unnamed: 4', 'Categorias', 'Unnamed: 6'],
      dtype='object')


In [67]:
df.columns = df.columns.str.replace('\n', ' ').str.strip()
df = df.rename(columns={
    'Unnamed: 4': 'Descrição quesito',
    'Categorias': 'Tipo categoria',
    'Unnamed: 6': 'Descrição categoria'
})
print(df.columns)

Index(['Posição inicial', 'Tamanho', 'Código da variável', 'Quesito',
       'Descrição quesito', 'Tipo categoria', 'Descrição categoria'],
      dtype='object')


Vamos seguir removendo as linhas que dividem a planilha em partes e módulo, além das linhas totalmente em branco, já que não serão relevantes para a análise:

In [ ]:
print(df.iloc[2]) # Exemplo de linha que separa a planilha

Posição inicial              NaN
Tamanho                      NaN
Código da variável           NaN
Quesito                       nº
Descrição quesito      descrição
Tipo categoria             Tipo 
Descrição categoria    Descrição
Name: 0, dtype: object


In [75]:
# Remove linhas onde 'Posição inicial' contém 'Parte' ou 'Módulo'
df = df[~df['Posição inicial'].astype(str).str.contains('Parte|Módulo', case=False, na=False)]
# Remove as linhas vazias
df = df.dropna(how='all')
print(df.iloc[2])

Posição inicial         NaN
Tamanho                 NaN
Código da variável      NaN
Quesito                 NaN
Descrição quesito       NaN
Tipo categoria           12
Descrição categoria    Acre
Name: 4, dtype: object


Algumas linhas ainda estão com informação faltando, e isso ocorre porque são mescladas com outras linhas. Para ajustar isso, vamos fazer um forward fill (Apenas das colunas específicas):

In [77]:
df = df.drop(index=0) # Linha com descrição da coluna

In [78]:
cols_to_ffill = ['Posição inicial', 'Tamanho', 'Código da variável', 'Quesito', 'Descrição quesito']
df[cols_to_ffill] = df[cols_to_ffill].ffill()
print(df.iloc[2])

Posição inicial                           1
Tamanho                                 2.0
Código da variável                    V0001
Quesito                                 NaN
Descrição quesito      Unidade da Federação
Tipo categoria                           13
Descrição categoria                Amazonas
Name: 5, dtype: object


Com os tratamentos feitos, vamos finalizar salvando o dicionário em um CSV para auxiliar na leitura dos microdados.

In [79]:
df.to_csv("dicionario_tratado.csv", index=False, encoding="utf-8")